# EI ST4

## Imports

In [ ]:
import pandas as pd
from datetime import datetime
import math

In [ ]:
df = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_train.csv")

## Cleaning up the dataframe

In [ ]:
df

Get unique count for each variable

In [ ]:
df.nunique()

Check variable data type

In [ ]:
df.dtypes

We can remove the `languageCode` and `countryCode` column as they are constant

In [ ]:
df[~df.languageCode.isin(['fr', 'FR'])].empty and df[~df.countryCode.eq('FRA')].empty

In [ ]:
df = df.drop(columns=['languageCode', 'countryCode'])

We will also remove the `userId`, `registrationDate`, `reactivationValue`, `emailContactable` and `postalContactable` as they are irrelevant

In [ ]:
df = df.drop(columns=['userId', 'registrationDate', 'reactivationValue', 'emailContactable', 'postalContactable'])

We are going to replace the `washDishes` `STRING` column with a `ownsDishwasher` `INT` column

In [ ]:
df['ownsDishwasher'] = df.apply(lambda row: 1 if row['washDishes'] == 'Auto' else -1, axis=1)
df = df.drop(columns=["washDishes"])

In the `MrPropreTrier`, `AntikalTrier`, `ArielTrier`, `DashTrier` we will replace `Known Trier` with `1` and `Nan` with `-1`

In [ ]:
for c in ['MrPropreTrier', 'AntikalTrier', 'ArielTrier', 'DashTrier']:
    df[c[:-5]] = df.apply(lambda row: 1 if row[c] == 'Known Trier' else -1, axis=1)
    df = df.drop(columns=[c])

We will replace `detergentType` by `liquidDetergent`

In [ ]:
df['liquidDetergent'] = df.apply(lambda row: 1 if row['detergentType'] == 'Liquid' else -1, axis=1)
df = df.drop(columns=['detergentType'])

Replacing `toothBrushType` with `electricToothbrush`

In [ ]:
df['electricToothbrush'] = df.apply(lambda row: 1 if row['toothBrushType'] == 'Electric' else -1, axis=1)
df = df.drop(columns=['toothBrushType'])

Replacing `petOwner` with `hasPet`, if the first 3 characters are 'Yes' or 'Oui' the value is `1`, else it is `-1`

In [ ]:
df['hasPet'] = df.apply(lambda row: 1 if row['petOwner'][:3] == 'Yes' or row['petOwner'][:3] == 'Oui' else -1, axis=1)
df = df.drop(columns=['petOwner'])

Transforms a `ISO 8601` time string to the amount of days since the time string

In [ ]:
def timeStringToDelta(timeString: str) -> int:
    return int((datetime.utcnow().timestamp() - datetime.strptime(timeString[:10], "%Y-%m-%d").timestamp()) / 3600 / 24)

Replaces `lastActivityDate` with `daysSinceActivity`: an `integer` representing the amount of ellapsed days since last activity

In [ ]:
df['daysSinceActivity'] = df.apply(lambda row: timeStringToDelta(row['lastActivityDate']), axis=1)
df = df.drop(columns=['lastActivityDate'])

In [ ]:
df

The possible values for `numberChildren` are `'1'`, `'2'`, `'3'`, `'4'`, `'4+'` and `'NaN'`. We will be creating a column `nbChildren` of type `int` where `'NaN'` will be mapped to `None`.

In [ ]:
def childrenMagik(children: str):
    if str(children).lower() in 'nan': return None
    if children == '4+': return 5
    return int(children)

df['nbChildren'] = df.apply(lambda row: childrenMagik(row['numberChildren']), axis=1)
df = df.drop(columns=['numberChildren'])

In [ ]:
df

In [ ]:
df['nbChildren'].mean()

In [ ]:
1.3804149665769598